# Script for converting haploid multiVCF to the FASTA

A consideration needs to be done here, the multi-vcf must be haploid and it accepts as max as three-allelic SNPs, if you have SNPs with more than three alleles you need to do little modification of the script in two points that are listed below.

In [ ]:
import allel
import os
import numpy as np

Only three alleles? you are fine, keep going. 

**Alert 1** First modification **if** you have more than three alleles for some SNPs: after the second `elif` you add 

``
elif s[i] == 3: 
    out.append(alt[i][2])
``

if you have four allele, than, if you have five alleles 

`
elif s[i] == 4: 
    out.append(alt[i][3])
` 

ecc. ecc.

This, needs to be followed by the modification in the import of the vcf file

In [ ]:
# Function for conversion of 0,1,2,"." in the ref and alt base
# It converts the haplotype array from numbers 0,1,2,. to the corresponding letter

def fromGT2Base(HaploArray):
    
    n=[]

    for sam in range(HaploArray.shape[1]):

        s= HaploArray[:,sam]

        out=[]

        for i in range(HaploArray.shape[0]):
            if s[i] == 0:
                out.append(ref[i])
            elif s[i] == 1:
                out.append(alt[i][0])
            elif s[i] == 2:
                out.append(alt[i][1])
            else:
                out.append("N")
        n.append(out)

    FASTA_array_tmp =np.array(n)
    return(FASTA_array_tmp)

Only three alleles? you are fine, keep going. 

**Alert 2** Second modification **if** you have more than three alleles for some SNPs: you need to change `numbers={'GT': 1, 'ALT':3}` if you have four alleles, then, if you have five alleles `numbers={'GT': 1, 'ALT':4}`, then if you have if you have six alleles `numbers={'GT': 1, 'ALT':5}` ecc. ecc.

In [ ]:
# Change directory
os.chdir('/media/ubuntu/26f3b6d7-0e54-4371-9efd-01b2f0f03216/project/TraversoetalYchrAmerica/make_fasta/')

# Load the vcf and take only 1 genotype (haploid) as well as the maximum two ALT alleles
callset = allel.read_vcf('merged_HC_130samps_complete_mm09_nomonomorph_corr.vcf.gz',  numbers={'GT': 1, 'ALT':2})

In [ ]:
# Show the keys of the callset
sorted(callset.keys())

In [ ]:
# Create the element for the ref / alt alleles and the ID of the samples
ref = callset['variants/REF']
ref
alt = callset['variants/ALT']
alt
IDsamp = callset['samples']
IDsamp

# Create the haplotype array from the callset of before
hap = allel.HaplotypeArray(callset['calldata/GT'])
hap

In [ ]:
# Vectorize in order to apply len function to the numpy array of the alt
mylen = np.vectorize(len)

# Create a vector with the length of each element of the array
l = mylen(alt)

# Filter only the length higher than 1 (i.e CC / CCA / CCCA)
higher_one_tmp1 = (l > 1)

# Create and empty array
higher_one_tmp2 = []

# If there is a True insert this in the new array
for el in range(higher_one_tmp1.shape[0]):
    higher_one_tmp2.append(any(higher_one_tmp1[el]))

# Revert the true and false    
higher_one = np.array([not elem for elem in higher_one_tmp2])

# Display the results
higher_one


In [ ]:
# Exclude those variants that had more than one base in the ref / alt
ref = ref[higher_one]
alt = alt[higher_one]
hap2 = hap[higher_one]

In [ ]:
FASTA_array_tmp = fromGT2Base(hap2)

In [ ]:
FASTA_array = FASTA_array_tmp.astype('S1')

In [ ]:
FASTA2save = FASTA_array[0]
allel.write_fasta("prova2.fasta", FASTA2save, names= IDsamp[0],mode='w', width=10000)

In [ ]:
for sa in range(1, FASTA_array.shape[0]):
    allel.write_fasta("prova2.fasta", FASTA_array[sa], names= IDsamp[sa],mode='a', width=10000)
    